In [1]:
import numpy as np
import pandas as pd

# Les données sont accessibles dans "/kaggle/input/".
# Ici, on affiche le contenu du dossier et des sous-dossiers

import os

data_dir = '/kaggle/input/inf8460-snli/'

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/inf8460-snli/snli_train.csv
/kaggle/input/inf8460-snli/snli_test.csv
/kaggle/input/inf8460-snli/sample_submission_snli.csv


In [2]:
# Lecture des données d'entraînement
train_data = pd.read_csv(os.path.join(data_dir, 'snli_train.csv'))
train_data.head()

,sentence1,sentence2,label1,id
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,neutral,0
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",contradiction,1
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",entailment,2
3,Children smiling and waving at camera,They are smiling at their parents,neutral,3
4,Children smiling and waving at camera,There are children present,entailment,4


In [3]:
# Lecture des données de test
test_data = pd.read_csv(os.path.join(data_dir, 'snli_test.csv'))
test_data.head()

,id,sentence1,sentence2
0,1000000,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...
1,1000001,Two women are embracing while holding to go pa...,Two woman are holding packages.
2,1000002,Two women are embracing while holding to go pa...,The men are fighting outside a deli.
3,1000003,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.
4,1000004,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.


In [4]:
# Ici, vous devez écrire votre modèle

# Le format idéalisé ressemblerait à :
#
# model = NLPModel
# model.learn(train_data)
# submission['score'] = model.predict(test_data)

# Dans ce notebook, on fait juste des prédictions aléatoires 
labels = train_data.label1.unique()

test_data["label1"] = np.random.choice(labels, size=len(test_data))
submission = test_data[["id", "label1"]]
submission.head()

,id,label1
0,1000000,entailment
1,1000001,neutral
2,1000002,neutral
3,1000003,entailment
4,1000004,neutral


In [5]:
# On sauvegarde nos prédictions dans un fichier
# On peut alors soumettre ce fichier à Kaggle pour l'évaluation
submission.to_csv('submission.csv', index=False)